# This one works

In [ ]:
!pip install openpyxl==3.0.0

In [75]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import LSTM, Dense
import plotly.graph_objects as go

In [ ]:

filename = "/content/drive/MyDrive/imdc/Irradiance_39.xlsx"
#filename = "Irradiance_39.xlsx"
df = pd.read_excel(filename)
print(df.info())

In [78]:
# Daily global horizontal solar irradiation (GHI) 
fields = ['Time', 'Ghi Prev Day', 'Ghi Curr Day'] # we only want to use these rows
filename = "/content/drive/MyDrive/imdc/Irradiance_39.xlsx"
df = pd.read_excel(filename, usecols=fields)
print(df.info)
#gc_df.set_index('Date', inplace=True)
#df.head(70)

<bound method DataFrame.info of        Time  Ghi Prev Day  Ghi Curr Day
0         1             0             0
1         2             0             0
2         3             0             0
3         4             0             0
4         5             0             0
...     ...           ...           ...
17515    44             0             0
17516    45             0             0
17517    46             0             0
17518    47             0             0
17519    48             0             0

[17520 rows x 3 columns]>


In [79]:
close_data = df['Ghi Curr Day'].values
close_data = close_data.reshape((-1,1))

split_percent = 0.80
split = int(split_percent*len(close_data))

close_train = close_data[:split]
close_test = close_data[split:]

date_train = df['Time'][:split]
date_test = df['Time'][split:]

print("Y values for training:", len(close_train))
print("Y values for testing:", len(close_test))
print("X values for training:", len(date_train))
print("X values for testing:", len(date_test))

Y values for training: 14016
Y values for testing: 3504
X values for training: 14016
X values for testing: 3504


In [80]:
look_back = 48 # 1 day on 30 min intervals

train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)
print("train", train_generator)
print("test", test_generator)

train <keras.preprocessing.sequence.TimeseriesGenerator object at 0x7f531d0c7710>
test <keras.preprocessing.sequence.TimeseriesGenerator object at 0x7f531d0c79d0>


In [81]:
model = Sequential()
model.add(
    LSTM(10,
        activation='relu',
        input_shape=(look_back,1))
)
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 25
model.fit(train_generator, epochs=num_epochs, verbose=1)

Epoch 1/25
699/699 [==============================] - 14s 19ms/step - loss: 363544520425472.0000
Epoch 2/25
699/699 [==============================] - 13s 18ms/step - loss: 516306.3125
Epoch 3/25
699/699 [==============================] - 13s 18ms/step - loss: 437989.8438
Epoch 4/25
699/699 [==============================] - 13s 18ms/step - loss: 549200.3750
Epoch 5/25
699/699 [==============================] - 13s 18ms/step - loss: 328686.1562
Epoch 6/25
699/699 [==============================] - 13s 19ms/step - loss: 99838.9531
Epoch 7/25
699/699 [==============================] - 13s 19ms/step - loss: 97384.1641
Epoch 8/25
699/699 [==============================] - 13s 19ms/step - loss: 96980.0312
Epoch 9/25
699/699 [==============================] - 13s 18ms/step - loss: 98029.8359
Epoch 10/25
699/699 [==============================] - 14s 20ms/step - loss: 97570.9766
Epoch 11/25
699/699 [==============================] - 14s 20ms/step - loss: 95459.5938
Epoch 12/25
699/699 [======

In [82]:
#import plotly.express as go
import plotly.graph_objects as go
prediction = model.predict(test_generator)
print(prediction)
print(len(prediction))

[[ 70.87949 ]
 [169.01309 ]
 [109.94898 ]
 ...
 [ 62.148083]
 [ 37.593464]
 [ 82.77896 ]]
3456


In [83]:
close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

print(len(close_train))
print(len(close_test))
print(len(prediction))
pred_dataSet = date_test[0:len(date_test)-look_back]

#fig = go.Figure()

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = pred_dataSet,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)

fig = go.Figure(data=[trace1, trace2, trace3])
fig.show()

14016
3504
3456


In [86]:
close_data = close_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = df['Time'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 30
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)
print("Prediction:", forecast)
print("Time:", forecast_dates)

Prediction: [ 0.         62.97915649 60.51493073 59.68458939 59.16721725 58.80222321
 58.52923584 58.31774139 58.15009308 58.01502991 57.90509796 57.81521225
 57.74175262 57.6820755  57.63426971 57.59677505 57.5775528  57.56230164
 57.53856659 57.51876831 57.50244904 57.48873138 57.91003036 60.47837067
 57.88611603 59.06352615 62.15164566 55.44221878 62.24905014 57.02282715
 41.3030777 ]
Time: [Timestamp('1970-01-01 00:00:00.000000048', freq='D'), Timestamp('1970-01-02 00:00:00.000000048', freq='D'), Timestamp('1970-01-03 00:00:00.000000048', freq='D'), Timestamp('1970-01-04 00:00:00.000000048', freq='D'), Timestamp('1970-01-05 00:00:00.000000048', freq='D'), Timestamp('1970-01-06 00:00:00.000000048', freq='D'), Timestamp('1970-01-07 00:00:00.000000048', freq='D'), Timestamp('1970-01-08 00:00:00.000000048', freq='D'), Timestamp('1970-01-09 00:00:00.000000048', freq='D'), Timestamp('1970-01-10 00:00:00.000000048', freq='D'), Timestamp('1970-01-11 00:00:00.000000048', freq='D'), Timestam

In [87]:

trace1 = go.Scatter(
    x = forecast_dates,
    y = forecast,
    mode = 'lines',
    name = 'Prediction'
)

fig = go.Figure(data=[trace1])
fig.show()